
# Project: heart failure clinical

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
This data set exploring the death case recodes and the attrubite for the patint and realted disease, the relationship with the patient who death or not, the data set provided from   <a href="https://www.kaggle.com/andrewmvd/heart-failure-clinical-data">kaggle</a>


## Question 
* Does the gender of the patient effect on likelihood of death case.
* Does the age give us single to probiblite to incress death case.
* Does the disease effect on death case.
* What disease effect on female more than male.
* The follow up help patient to survival.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import plotly.figure_factory as ff


from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import xgboost



In [ ]:
df= pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.describe()

# Data Wrangling

####Before start let describe the value for the attributes
#### 0 mean the patient have not disease, 1 mean the patient disease
* Age: the age for patient.
* Anaemia: Decrease of red blood cells or hemoglobin,
* Creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
* Diabetes: If the patient has diabetes 
* Ejection_fraction: Percentage of blood leaving the heart at each contraction (percentage)
* High_blood_pressure: If the patient has hypertension (boolean)
* platelets: Platelets in the blood (kiloplatelets/mL)
* Serum_creatinine:  Level of serum creatinine in the blood (mg/dL)
* serum_sodium: Level of serum sodium in the blood (mEq/L)
* Sex: Woman or man (binary)
* Smoking: If the patient smokes or not (boolean)
* Time: Follow-up period (days)
* DEATH_EVENT: If the patient deceased during the follow-up period "our Goals"(boolean)


In [ ]:
correlation = df.corr()
plt.figure(figsize=(16,12))
plt.title('Correlation Heatmap of Death event')
ax = sns.heatmap(correlation, square=True, annot=True, fmt='.2f', linecolor='white')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_yticklabels(ax.get_yticklabels(), rotation=30)           
plt.show()

#EDA

#Age 
when happened the death case, we look to age for the patient, So let look inside the age attribute and correlation with another attribute.

In [ ]:
fig = px.histogram(x=df['age'], nbins=50, histnorm='density')
fig.update_layout(title='Age distribution:',
                 xaxis_title='age', yaxis_title='Count')

In [ ]:
g = sns.catplot(x="age", kind="box",data=df)

In [ ]:
fig = px.box(df, x='DEATH_EVENT', y='age',color='sex')
fig.update_layout(
    title_text="Gender wise Age Spread and Death - Male = 1 Female =0")
fig.show()

In [ ]:
ax = sns.barplot(x=df.sex, y= df.age,hue=df.DEATH_EVENT, data=df)
ax.set_title("Death age for Females and Males")
x_ticks_labels=['Female', 'Male']
ax.set_xticklabels(x_ticks_labels)
plt.show()

From the above chart we noted, the mean for age around 60, the common rang for age from 60 to 70, we have extreme value in 94-95 and it's an outlier for the female gender, the average death case for a male is 65 and the average death case for a female is 60, this value it's around the mean for age at all, So here we not have any strong correlation.
##let care more in gender, maybe will find the reason behind male death more than female.
##note: the number of male recored more than female, So maybe find differences
#Gender

In [ ]:
ax = sns.countplot(x=df.sex, hue=df.DEATH_EVENT, data=df)
ax.set_title("Death case for Females and Males")
x_ticks_labels=['Female', 'Male']
ax.set_xticklabels(x_ticks_labels)
plt.show()

based on above bar chart we noticed the male record more than female, So the analysis will baise to male more than female, but let look in depth for both gender.

In [ ]:
gender_smoking= df.groupby('sex')['smoking'].sum().sort_values(ascending=False)
fig = px.bar(df,gender_smoking.index, gender_smoking)
fig.update_layout(title='Relationship between gender and smoking - Male = 1 Female =0')
fig.show()

In [ ]:
gender_smoking= df.groupby('sex')['high_blood_pressure'].sum().sort_values(ascending=False)
fig = px.bar(df,gender_smoking.index, gender_smoking)
fig.update_layout(title='high blood_pressure for both gender - Male = 1 Female =0')
fig.show()

In [ ]:
gender_smoking= df.groupby('sex')['diabetes'].sum().sort_values(ascending=False)
fig = px.bar(df,gender_smoking.index, gender_smoking)
fig.update_layout(title='diabetes for both gender - Male = 1 Female =0')
fig.show()

In [ ]:
gender_smoking= df.groupby('sex')['creatinine_phosphokinase'].sum().sort_values(ascending=False)
fig = px.bar(df,gender_smoking.index, gender_smoking)
fig.update_layout(title='high blood pressure for both gender')
fig.show()

After study the analysis for both gender, I've found the male smoking than female, that give a Signal for the smoking male More likely for the infection in diseases and death more early

In [ ]:
male = df[df["sex"]==1]
female = df[df["sex"]==0]

male_survi = male[df["DEATH_EVENT"]==0]
male_not = male[df["DEATH_EVENT"]==1]
female_survi = female[df["DEATH_EVENT"]==0]
female_not = female[df["DEATH_EVENT"]==1]

labels = ['Male - Survived','Male - Not Survived', "Female -  Survived", "Female - Not Survived"]
values = [len(male[df["DEATH_EVENT"]==0]),len(male[df["DEATH_EVENT"]==1]),
         len(female[df["DEATH_EVENT"]==0]),len(female[df["DEATH_EVENT"]==1])]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Analysis on Survival - Gender")
fig.show()

In [ ]:


ax = sns.barplot(x=df.sex, y= df.platelets,hue=df.DEATH_EVENT, data=df)
ax.set_title("Death age for Females and Males")
x_ticks_labels=['Female', 'Male']
ax.set_xticklabels(x_ticks_labels)
plt.show()

To know the relationship between Platelets and death, I do research on the internet, I've found, low <a href="https://www.rnceus.com/cbc/cbcplate.html">platelets</a> to death case, if we look to the graph we find the female who Survival have high platelets more than female have low platelets, but in male records death case in both Are equal
#Creatinine phosphokinase


In [ ]:


fig = px.scatter(df, x="creatinine_phosphokinase", y="age", color="DEATH_EVENT", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()


In [ ]:
fig = px.box(df, x='DEATH_EVENT', y='creatinine_phosphokinase')
fig.update_layout(
    title_text="Gender wise Age Spread and Death - Male = 1 Female =0")
fig.show()

In [ ]:


ax = sns.barplot(x=df.sex, y= df.creatinine_phosphokinase,hue=df.DEATH_EVENT, data=df)
ax.set_title("Death age for Females and Males")
x_ticks_labels=['Female', 'Male']
ax.set_xticklabels(x_ticks_labels)
plt.show()

The <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4877131/
">study</a> shows the patient who has a high level of Creatinine phosphokinase it's more likelihood to death if we look to your graph will find same result but our sample is too small to find, also i've found the pation older than 60  more likleyhood to death with high Creatinine phosphokinase, the death case in male have high level of CP.
# Serum creatinine


In [ ]:




fig = px.scatter(df, x="serum_creatinine", y="age", color="DEATH_EVENT", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()


In [ ]:

ax = sns.barplot(x=df.sex, y= df.serum_creatinine,hue=df.DEATH_EVENT, data=df)
ax.set_title("Death age for Females and Males")
x_ticks_labels=['Female', 'Male']
ax.set_xticklabels(x_ticks_labels)
plt.show()

Female patients more likely to die due to high serum creatinine in blood, High levels of SC lead to an increase of death cases with an older patient.
# Time

In [ ]:
folowUp = df.groupby('time').sum()['DEATH_EVENT']
folowUp.plot(figsize=(20,10),title='trends on death times as folow up increased')

the high death case happened in the first follow up time, the state of patients more likely to be stable with time. 

# Model Building



In [ ]:

x=df.drop(columns=('DEATH_EVENT'))
y=df['DEATH_EVENT']
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=2)

# RandomForestClassifier

In [ ]:

clf = RandomForestClassifier(max_depth=2, random_state=1)
clf.fit(x_train, y_train)
pred=clf.predict(x_test)
clf.score(x_test,y_test)

# Confusion matrix for Random forest 

In [ ]:
cm = confusion_matrix(y_test, pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title("Random Forest Model - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

# GradientBoostingClassifier


In [ ]:
gradientboost_clf = GradientBoostingClassifier(max_depth=2, random_state=1)
gradientboost_clf.fit(x_train,y_train)
gradientboost_pred = gradientboost_clf.predict(x_test)

In [ ]:
gradientboost_clf.score(x_test,y_test)

# Confusion matrix for GradientBoostingClassifier

In [ ]:
cm = confusion_matrix(y_test, gradientboost_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title("Gredient Boosting Model - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

# Xgbrf classifier


In [ ]:
# xgbrf classifier

xgb_clf = xgboost.XGBRFClassifier(max_depth=2, random_state=1)
xgb_clf.fit(x_train,y_train)
xgb_pred = xgb_clf.predict(x_test)
xgb_clf.score(x_test,y_test)

# Confusion matrix Xgbrf classifier


In [ ]:
cm = confusion_matrix(y_test, xgb_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title("XGBRFClassifier Model - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

#  Lightgbm 


In [ ]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=2, random_state=1)
lgb_clf.fit(x_train,y_train)
lgb_pred = lgb_clf.predict(x_test)
lgb_clf.score(x_test,y_test)

# Confusion matrix for Lightgbm

In [ ]:
cm = confusion_matrix(y_test, lgb_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title("LGBMClassifier Model - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

# LImitations
* The data size is too small only 299 records.
* we do not know the main reason for death case, The patient died due to illness or accident.
* the sample force on male more than female, Do we not know if the female is more likely to die than male.
* the time attribute not clear it's for day or month or year, or for each dr 
visit.
* The data have a high number of an outlier in attribute 

# Conclusions
* Age, creatinine_phosphokinase, serum_creatinine, Time Strong attribute effect on death event.
* The male has a high death number.
* The female have high serum creatinine levels have high likely to death.
* the follow up in long term decrease death case.
* If we care on the patient who has low anemia, high creatinine phosphokinase, and  serum creatinine, the death percentage will decrease